In [0]:
import re
import numpy as np 

In [2]:
with open('truyenkieu.txt') as f:
  text = f.read()

text[:30]

'1..Trăm năm trong cõi người ta'

In [3]:
text = re.sub(r'[0-9\.]', '', text)
text[:30]

'Trăm năm trong cõi người ta,\n\n'

In [0]:
chars = list(set(text))
char2int = {c:i for i,c in enumerate(chars)}
int2char = {i:c for i,c in enumerate(chars)}

In [0]:
int_text = np.array([char2int[c] for c in text])

In [0]:
val_size = int(0.1*len(int_text))
val_int_text, int_text = int_text[:val_size], int_text[val_size:]

In [7]:
print(f"Number chars in text : {len(text)}")
print(f"Number unique chars in text : {len(chars)}")

Number chars in text : 107561
Number unique chars in text : 128


In [8]:
def one_hot_encode(arr, vocab_size):
  x = np.zeros((arr.size, vocab_size), np.float32)
  x[np.arange(arr.size), arr.flatten()] = 1
  x = x.reshape((*arr.shape, vocab_size))
  return x 

arr = np.array([[5, 3, 2],
       [1, 1, 1]])

print(one_hot_encode(arr, 6))

[[[0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]]]


In [0]:
def generate_batches(arr, batch_size, seq_len):
  full_batch = batch_size*seq_len
  num_batches = len(arr)//full_batch 
  arr = arr[:num_batches*full_batch]
  arr = arr.reshape((batch_size, -1))
  for i in range(0, arr.shape[1], seq_len):
    x = arr[:, i:i+seq_len]
    y = np.zeros_like(x)
    try:
      y[:, :-1], y[:, -1] = x[:, 1:], arr[:, i+seq_len+1]
    except IndexError:
      y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
    yield x, y

In [10]:
x, y = next(iter(generate_batches(int_text, 3, 10)))
print(x)
print("---------------------")
print(y)

[[ 56  31  59  50  75  11  92  50  96 101]
 [ 80  50  10  39  50 126 124 116  31  50]
 [ 50  42  52  31  59  50  75  76  92  50]]
---------------------
[[ 31  59  50  75  11  92  50  96 101  42]
 [ 50  10  39  50 126 124 116  31  50  92]
 [ 42  52  31  59  50  75  76  92  50  92]]


In [19]:
import torch 
import torch.nn as nn 

class Network(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, use_one_hot):
    super().__init__()
    self.hidden_dim = hidden_dim 
    self.n_layers = n_layers
    self.use_one_hot = use_one_hot
    if not use_one_hot:
      self.embedding = nn.Embedding(vocab_size, embedding_dim)
      self.LSTM = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=0.5, batch_first=True)
    else:
      self.LSTM = nn.LSTM(vocab_size, hidden_dim, n_layers, dropout=0.5, batch_first=True)
    self.dropout = nn.Dropout(0.5)
    self.fc = nn.Linear(hidden_dim, vocab_size)
  
  def forward(self, x, hidden):
    if not self.use_one_hot:
      x = self.embedding(x)
    x, hidden = self.LSTM(x, hidden)
    x = x.contiguous().view(-1, self.hidden_dim)
    x = self.dropout(x)
    x = self.fc(x)
    return x, hidden

  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data
    hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
              weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
    return hidden 

device = "cuda" if torch.cuda.is_available() else "cpu"

vocab_size = len(chars)
embedding_dim = 100
hidden_dim = 512
n_layers = 2

one_hot_model = Network(vocab_size, embedding_dim, hidden_dim, n_layers, use_one_hot=True).to(device)
embedding_model = Network(vocab_size, embedding_dim, hidden_dim, n_layers, use_one_hot=False).to(device)
print(one_hot_model)
print(embedding_model)

Network(
  (LSTM): LSTM(128, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=128, bias=True)
)
Network(
  (embedding): Embedding(128, 100)
  (LSTM): LSTM(100, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=128, bias=True)
)


In [0]:
import torch.optim as optim 

def train_model(model, use_one_hot):

  if use_one_hot:
    print("Model use one hot encoded ~~~")
  else:
    print("Model use embedding layer !!!")
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  epochs = 30
  batch_size = 20
  seq_len = 100
  clip = 5
  min_val_loss = np.Inf

  val_idx = int(0.1*len(int_text))
  train, val_data = int_text[:-val_idx], int_text[-val_idx:]
  model.train()
  for e in range(epochs):
    h = model.init_hidden(batch_size)
    for x, y in generate_batches(train, batch_size, seq_len):
      optimizer.zero_grad()
      if use_one_hot:
        x = one_hot_encode(x, vocab_size)
      x, y = torch.from_numpy(x).to(device), torch.from_numpy(y).to(device)
      h = tuple([e.data for e in h])
      out, h = model(x, h)
      loss = criterion(out, y.view(batch_size*seq_len).long())
      loss.backward()
      nn.utils.clip_grad_norm_(model.parameters(), clip)
      optimizer.step()
    
    else:
      model.eval()
      val_losses = []
      # with torch.no_grad():
      val_h = model.init_hidden(batch_size)
      for x, y in generate_batches(val_data, batch_size, seq_len):
        if use_one_hot:
          x = one_hot_encode(x, vocab_size)
        x, y = torch.from_numpy(x).to(device), torch.from_numpy(y).to(device)
        val_h = tuple([e.data for e in h])
        out, h = model(x, h)
        val_loss = criterion(out, y.view(batch_size*seq_len).long())
        val_losses.append(val_loss.item())
      print("Epoch {}/{}.\tTraining loss : {:6.4f}\tValidation loss : {:6.4f}".format(e+1, epochs, loss.item(), np.mean(val_losses)))
      if val_loss < min_val_loss:
        print("Validation loss decrease {:6.4f} ----> {:6.4f}.".format(min_val_loss, val_loss))
        min_val_loss = val_loss
        print("Save weights...")
        torch.save(model.state_dict(), f'model_{int(use_one_hot)}.h5')
      model.train()
  print("------------------------------------------------")
  return model 

In [21]:
one_hot_model = train_model(one_hot_model, use_one_hot=True)
embedding_model = train_model(embedding_model, use_one_hot=False)

Model use one hot encoded ~~~
Epoch 1/30.	Training loss : 3.5307	Validation loss : 3.5280
Validation loss decrease    inf ----> 3.5320.
Save weights...
Epoch 2/30.	Training loss : 3.4254	Validation loss : 3.4126
Validation loss decrease 3.5320 ----> 3.4144.
Save weights...
Epoch 3/30.	Training loss : 2.7916	Validation loss : 2.7273
Validation loss decrease 3.4144 ----> 2.7283.
Save weights...
Epoch 4/30.	Training loss : 2.4426	Validation loss : 2.3888
Validation loss decrease 2.7283 ----> 2.3916.
Save weights...
Epoch 5/30.	Training loss : 2.2849	Validation loss : 2.2232
Validation loss decrease 2.3916 ----> 2.2177.
Save weights...
Epoch 6/30.	Training loss : 2.1614	Validation loss : 2.1115
Validation loss decrease 2.2177 ----> 2.1007.
Save weights...
Epoch 7/30.	Training loss : 2.0658	Validation loss : 2.0272
Validation loss decrease 2.1007 ----> 2.0157.
Save weights...
Epoch 8/30.	Training loss : 1.9969	Validation loss : 1.9546
Validation loss decrease 2.0157 ----> 1.9393.
Save weigh

In [0]:
import torch.nn.functional as F
def predict(net, char, use_one_hot, h=None, top_k=None):

  x = np.array([[char2int[char]]])
  if use_one_hot:
    x = one_hot_encode(x, len(chars))
  inputs = torch.from_numpy(x).to(device)
  h = tuple([each.data for each in h])
  out, h = net(inputs, h)
  p = F.softmax(out, dim=1).data
  p = p.to("cpu")

  if top_k is None:
    top_ch = np.arange(len(chars))
  else:
    p, top_ch = p.topk(top_k)
    top_ch = top_ch.numpy().squeeze()
  
  p = p.numpy().squeeze()
  char = np.random.choice(top_ch, p=p/p.sum())

  return int2char[char], h 

In [0]:
def sample(net, size, use_one_hot, prime='The', top_k=None):
  net.to(device)
  net.eval()
  chars = [ch for ch in prime]
  h = net.init_hidden(1)
  for ch in prime:
    char, h = predict(net, ch, use_one_hot, h, top_k=top_k)
  
  chars.append(char)

  for ii in range(size):
    char, h = predict(net, char[-1], use_one_hot, h, top_k=top_k)
    chars.append(char)
  
  return ''.join(chars)

In [25]:
embedding_model.load_state_dict(torch.load('./model_0.h5'))
print(sample(embedding_model, 1000, use_one_hot=False, prime='Trăm năm', top_k=5))

Trăm năm ngọc nào đường

 Thiệt cho đã dễ ngọn ngày,

Thôi cho ngọc thế tiếc thư lạ chàng

 Ngoại lời thế thấy thanh mây,

 Những nghiều nghiên thấy một lời vào này

Nỗi ra thì chẳng ngang người,

 Đươm đường ngọn mới bạc vào chuyên trên

Thương nhau nhắn nghĩa đến tay,

Mày thương chẳng một người cho thế thông

Nghĩ trời đá đã đã trong,

 Biết tình đài mới trước tay trước ngoài 

Tiếp đà thiếp chẳng mưa đường,

Một đâu mà đã bán mày chẳng ngườ

 Bốn tay như nói trong dâo,

 Nàng điều cho chút như châu Thôi chăng:

Nghe đưa đã bạc chưa ngươi,

Người này những đã biết đầu thấy chinh?

 Nghĩ cha đã trước cho lời,

Một trời nàng cũng chi ngày thư ra 

 Đường thuyền lại nghĩ ngày ta,

Nhìn đường thì cũng cho đâu chẳng người:

Chiều rướm trăm đức đã đường,

 Thương càng cũng tiếng cho cho cho trường ?

Buồn chinh lại đã đổn thuênh,

 Như rời thân đã cửa ngay thế thì!

Thấp người thiểp một mặt may,

Mà cho chẳng nguyệt đã chàng tinh hoa

Thúy tinh thấp có đến thưa,

Một ngay đã nghìn ngọn đà

In [29]:
one_hot_model.load_state_dict(torch.load('./model_1.h5'))
print(sample(one_hot_model, 1000, use_one_hot=True, prime='Trăm năm', top_k=5))

Trăm năm

Nghe rời được mặt ngoại trời,

Một người, nhi nói thưa ra chẳng lờ

Ngọn thư buồn ngận thế nào,

Tho quân thuy cát mười vào cửa như?

Thương đao thanh mới thì tha,

Thưa cho đã chẳng chẳng là cũng nhâu

 Thôi trời đánh nhận, trăng chàng,

Thương nghe trước đánh, mai trời có tay

 Đã lầu được người thanh đây,

 Thanh cho thong một nói đàu nhuy ta 

Chàng công nghe nỗi tin hoa,

 Nỗi lời, cũng ngắm một nha trong tì nên 

Biếc người lướm đã trăm người,

Trong lòng chong chẳng cũng thai trong ngay

Nghĩ triêm tan thúc thi hồng,

Thanh cho đã bấy như thì đết đan !

Đã thương thì nỗi những tiền,

Chẳng cho có mách thiệng lời ngày ngươi 

 Đau thanh đã chén cho thân,

 Còn cang nàng chiệm trưng trong thước trăg 

Chẳng như chẳng thút tinh càng,

 Ba đây cho đã ngọi canh đám triêng

Người về trăm chẳng trăn cào,

 Thong đân mười má lại đồi nghe nơi

 Chàng cho người tiếc thanh thang,

Nghĩ tin nhà cát mang đà lấn tam

Thiều thay một ngọc thì thười,

Một mâu nghĩ đáng mấy mình là ngườ